In [1]:
import pandas as pd
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

import numpy as np

from linearmodels.panel import PanelOLS
import datetime as dt

# Characteristics OLS

Professor said we should start with a regular OLS Regression.
I quickly looked into this and think a random effects panel regression would probably fit our data better (still need to read more into this)
So probably we can first do a regular OLS and than compare it to the random effects panel regression. Just some initial thoughts though!!!
Here is some intial code to run them:


In [18]:
#decide which df to use
df = pd.read_csv("Dataframes/macro.csv")
#df = df.loc[:, ["Instrument", "Date", "Earnings Per Share - Actual Surprise", "Revenue - Actual", "Net Income after Tax"]]
X_var_names = ["WACC Inflation Adjusted Risk Free Rate, (%)", "Unemployment rate"]
df

,Instrument,Date,GICS Industry Group Name,Earnings Per Share - Actual Surprise,"WACC Inflation Adjusted Risk Free Rate, (%)",Unemployment rate
0,AVY.N,2013-01-01,Materials,11.178,NaN,8.0
1,AVY.N,2013-04-01,Materials,2.482,NaN,7.6
2,AVY.N,2013-07-01,Materials,1.068,NaN,7.3
3,AVY.N,2013-10-01,Materials,8.095,NaN,7.2
4,AVY.N,2014-01-01,Materials,1.471,NaN,6.6
...,...,...,...,...,...,...
20115,POOL.OQ,2021-10-01,Retailing,17.194,1.527139,4.5
20116,POOL.OQ,2022-01-01,Retailing,40.267,1.515266,4.0
20117,POOL.OQ,2022-04-01,Retailing,34.342,2.325202,3.6
20118,POOL.OQ,2022-07-01,Retailing,1.503,3.092855,3.5


#### OLS Regression

In [19]:
#identifying outliers and replacing them with NA
summary_stats = df["Earnings Per Share - Actual Surprise"].describe()
Q1 = summary_stats.loc['25%']
Q3 = summary_stats.loc['75%']
IQR = Q3 - Q1
threshold = 7 #1.5 is standard threshold but we still want to keep enough variation in the data so setting threshol higher here
surprise_outliers_removed = df["Earnings Per Share - Actual Surprise"].loc[~((df["Earnings Per Share - Actual Surprise"] < (Q1 - threshold * IQR)) | (df["Earnings Per Share - Actual Surprise"] > (Q3 + threshold * IQR)))]
df_accuracy_new = df.copy()
df_accuracy_new["Earnings Per Share - Actual Surprise"] = surprise_outliers_removed
df_accuracy_new = df_accuracy_new.dropna(subset=["Earnings Per Share - Actual Surprise"])
#df_accuracy_new["Recommendation - Mean (1-5)"] = df_accuracy_new["Recommendation - Mean (1-5)"].fillna(0)
df_accuracy_new = df_accuracy_new.dropna()


In [20]:
df_accuracy_new

,Instrument,Date,GICS Industry Group Name,Earnings Per Share - Actual Surprise,"WACC Inflation Adjusted Risk Free Rate, (%)",Unemployment rate
12,AVY.N,2016-01-01,Materials,8.817,2.304994,4.8
13,AVY.N,2016-04-01,Materials,9.159,1.829833,5.1
14,AVY.N,2016-07-01,Materials,7.522,1.491713,4.8
15,AVY.N,2016-10-01,Materials,1.290,1.606498,4.9
16,AVY.N,2017-01-01,Materials,6.180,2.431507,4.7
...,...,...,...,...,...,...
20115,POOL.OQ,2021-10-01,Retailing,17.194,1.527139,4.5
20116,POOL.OQ,2022-01-01,Retailing,40.267,1.515266,4.0
20117,POOL.OQ,2022-04-01,Retailing,34.342,2.325202,3.6
20118,POOL.OQ,2022-07-01,Retailing,1.503,3.092855,3.5


In [21]:
y = df_accuracy_new["Earnings Per Share - Actual Surprise"]
#X = df_accuracy_new[['Revenue - Actual', 'Net Income after Tax']]
X = df_accuracy_new[X_var_names]
X = sm.add_constant(X)

model = sm.OLS(y, X).fit()

model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                     OLS Regression Results                                     
================================================================================================
Dep. Variable:     Earnings Per Share - Actual Surprise   R-squared:                       0.015
Model:                                              OLS   Adj. R-squared:                  0.015
Method:                                   Least Squares   F-statistic:                     95.66
Date:                                  Sun, 26 Feb 2023   Prob (F-statistic):           5.76e-42
Time:                                          18:53:59   Log-Likelihood:                -54926.
No. Observations:                                 12860   AIC:                         1.099e+05
Df Residuals:                                     12857   BIC:                         1.099e+05
Df Model:                                             2                                         
Covariance Type:                              nonrobust                                         
===============================================================================================================
                                                  coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------------------
const                                          13.4859      0.901     14.965      0.000      11.720      15.252
WACC Inflation Adjusted Risk Free Rate, (%)    -2.8010      0.264    -10.590      0.000      -3.319      -2.283
Unemployment rate                              -0.0142      0.089     -0.160      0.873      -0.188       0.160
==============================================================================
Omnibus:                     2185.142   Durbin-Watson:                   1.628
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            19558.989
Skew:                           0.554   Prob(JB):                         0.00
Kurtosis:                       8.939   Cond. No.                         35.5
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

#### OLS Pooled Regression

In [22]:
df_accuracy_new['Date'] = pd.to_datetime(df_accuracy_new['Date'], infer_datetime_format=True)
df_accuracy_new.dtypes

Instrument                                             object
Date                                           datetime64[ns]
GICS Industry Group Name                               object
Earnings Per Share - Actual Surprise                  float64
WACC Inflation Adjusted Risk Free Rate, (%)           float64
Unemployment rate                                     float64
dtype: object

In [23]:
#df_accuracy_new.set_index(["Instrument", "Date"], inplace=True)
df_accuracy_new

,Instrument,Date,GICS Industry Group Name,Earnings Per Share - Actual Surprise,"WACC Inflation Adjusted Risk Free Rate, (%)",Unemployment rate
12,AVY.N,2016-01-01,Materials,8.817,2.304994,4.8
13,AVY.N,2016-04-01,Materials,9.159,1.829833,5.1
14,AVY.N,2016-07-01,Materials,7.522,1.491713,4.8
15,AVY.N,2016-10-01,Materials,1.290,1.606498,4.9
16,AVY.N,2017-01-01,Materials,6.180,2.431507,4.7
...,...,...,...,...,...,...
20115,POOL.OQ,2021-10-01,Retailing,17.194,1.527139,4.5
20116,POOL.OQ,2022-01-01,Retailing,40.267,1.515266,4.0
20117,POOL.OQ,2022-04-01,Retailing,34.342,2.325202,3.6
20118,POOL.OQ,2022-07-01,Retailing,1.503,3.092855,3.5


# with grouping


In [24]:
from statsmodels.iolib.summary2 import summary_col
# Split the DataFrame into groups based on the stocks
df = df_accuracy_new[df_accuracy_new['GICS Industry Group Name'] == 'Technology Hardware & Equipment']
groups = df.groupby('Instrument')

# Define a function to perform OLS regression on each group
def ols_regression(group):
    # Define the dependent and independent variables
    if len(group) < 2:
        return None
    y = group['Earnings Per Share - Actual Surprise']
    X = group[X_var_names]

    # Add a constant to the independent variables
    X = sm.add_constant(X)

    # Fit the OLS model and return the results
    model = sm.OLS(y, X).fit()
    return model

# Apply the function to each group of data
results = groups.apply(ols_regression)
results = results.dropna()
# Collect the results into a new DataFrame

# Print the coefficients, t-values, and p-values
# print('Coefficients:')
# print(coefficients)
# print('\nT-values:')
# print(t_values)
# print('\nP-values:')
models = results.tolist()
summary = summary_col(models, stars=True, float_format='%0.2f', model_names=results.index.tolist())

# Print the combined summary table
summary

,AAPL.OQ,ANET.N,APH.N,CDW.OQ,CSCO.OQ,FFIV.OQ,GLW.N,HPE.N,HPQ.N,JNPR.N,KEYS.N,MSI.N,NTAP.OQ,STX.OQ,TDY.N,TEL.N,TRMB.OQ,WDC.OQ,ZBRA.OQ
const,11.13,0.21,16.99**,20.02***,4.58,-2.74,4.63,18.35,16.32,2.65,32.83**,3.94,17.03,41.33,25.06,-0.59,16.80*,55.93**,17.81*
,(10.69),(10.23),(7.43),(5.86),(3.81),(4.93),(5.16),(15.33),(11.33),(8.33),(13.49),(7.97),(16.86),(28.96),(14.84),(5.45),(9.32),(21.97),(10.34)
"WACC Inflation Adjusted Risk Free Rate, (%)",-3.72,5.61*,-3.43,-4.66**,-0.29,1.65,-1.75,-0.78,-6.01*,2.67,-3.82,1.12,-6.14,-18.07**,-2.46,-3.06,-6.38**,-17.53**,-3.41
,(3.15),(3.01),(2.19),(1.72),(0.68),(1.45),(1.52),(4.56),(3.36),(2.45),(3.97),(2.35),(4.96),(4.32),(4.37),(2.13),(2.72),(6.47),(3.04)
Unemployment rate,0.86,0.55,-0.34,-0.63,-0.43,0.97*,1.03*,-1.30,0.28,-0.51,-2.94**,0.97,1.81,-2.60,-0.98,2.74***,1.40,-3.07,-0.80
,(1.03),(0.98),(0.71),(0.56),(0.61),(0.47),(0.52),(1.50),(1.11),(0.80),(1.30),(0.77),(1.62),(5.33),(1.43),(0.72),(0.87),(2.11),(0.99)
R-squared,0.20,0.15,0.11,0.25,0.11,0.15,0.35,0.04,0.20,0.15,0.19,0.07,0.25,0.91,0.02,0.62,0.54,0.23,0.05
R-squared Adj.,0.14,0.08,0.04,0.19,-0.33,0.08,0.30,-0.04,0.13,0.09,0.12,-0.01,0.19,0.85,-0.06,0.59,0.50,0.17,-0.03
